
# 🕵️ SQL Murder Mystery — Resolución por Dos Caminos

Este notebook resuelve el famoso asesinato en SQL City del 15 de enero de 2018, usando dos enfoques diferentes:

1. 🔎 **Enfoque 1: Llegando directamente al cerebro pensante**
2. 💡 **Enfoque 2: Pasando por el asesino primero**

Todos los pasos están aplicados a la base de datos local: `./data/sql_murder_mistery.db`, que replica la web del juego.

---


## 🛠️ Conexión a la base de datos

In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("./data/sql-murder-mystery.db")

## 🔍 Enfoque 1

### Paso 1: Leer el informe del crimen

In [10]:
query = '''
SELECT * FROM crime_scene_report
WHERE date = "20180115"
    AND type = "murder" 
    AND city = "SQL City"
'''
df = pd.read_sql_query(query, conn)
df

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [11]:
print(df["description"][0])

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


### Paso 2: Buscamos a Annabel con los datos que sabemos

In [17]:
query = '''
SELECT * FROM person
WHERE address_street_name = "Franklin Ave"
'''
pd.read_sql_query(query, conn)

,id,name,license_id,address_number,address_street_name,ssn
0,12207,Wilmer Wolever,509484,139,Franklin Ave,636825374
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143
2,17683,Johnnie Schee,968887,1277,Franklin Ave,815977821
3,18651,Carleen Etoll,356746,22,Franklin Ave,193369255
4,22636,Zachary Ybarbo,768359,785,Franklin Ave,285346605
5,24737,Gema Nantz,273410,3968,Franklin Ave,180545802
6,30654,Clarita Rickels,418084,2254,Franklin Ave,714941023
7,32264,Shelby Dezeeuw,735415,1391,Franklin Ave,143197463
8,33793,Amado Mattan,161915,99,Franklin Ave,125205748
9,34592,Cordell Lindamood,592762,3657,Franklin Ave,509890333


### Paso 3: Sabiendo sus datos, buscamos en sus eventos de Facebook

In [18]:
query = '''
SELECT a.id, b.*
FROM person AS a
LEFT JOIN facebook_event_checkin AS b
ON a.id = b.person_id
WHERE id = 16371
'''
pd.read_sql_query(query, conn)

,id,person_id,event_id,event_name,date
0,16371,16371,4719,The Funky Grooves Tour,20180115


### Paso 4: Buscamos en interview

In [20]:
query = '''
SELECT a.id, b.*
FROM person AS a
LEFT JOIN interview AS b
ON a.id = b.person_id
WHERE id = 16371
'''
df = pd.read_sql_query(query, conn)
df

,id,person_id,transcript
0,16371,16371,"I saw the murder happen, and I recognized the ..."


In [21]:
print(df["transcript"][0])

I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


### Paso 5: Buscamos en la tabla de gym basándonos en el testimonio

In [22]:
query = '''
SELECT a.id, b.*
FROM person AS a
JOIN get_fit_now_member AS b
ON a.id = b.person_id
WHERE a.id = 16371
'''
pd.read_sql_query(query, conn)

,id,id,person_id,name,membership_start_date,membership_status
0,16371,90081,16371,Annabel Miller,20160208,gold


### Paso 6: Deben haber entrado a la vez, es el caso de Jeremy Bowers y Joe Germuska

In [25]:
query = '''
SELECT a.person_id, a.name, b.*
FROM get_fit_now_member AS a
JOIN get_fit_now_check_in AS b
ON a.id = b.membership_id
WHERE b.check_in_date = 20180109
'''
pd.read_sql_query(query, conn)

,person_id,name,membership_id,check_in_date,check_in_time,check_out_time
0,15247,Shondra Ledlow,X0643,20180109,957,1164
1,28073,Zackary Cabotage,UK1F2,20180109,344,518
2,55662,Sarita Bartosh,XTE42,20180109,486,1124
3,10815,Adriane Pelligra,1AE2H,20180109,461,944
4,83186,Burton Grippe,6LSTG,20180109,399,515
5,31523,Blossom Crescenzo,7MWHJ,20180109,273,885
6,92736,Carmen Dimick,GE5Q8,20180109,367,959
7,28819,Joe Germuska,48Z7A,20180109,1600,1730
8,67318,Jeremy Bowers,48Z55,20180109,1530,1700
9,16371,Annabel Miller,90081,20180109,1600,1700


### Paso 7: Buscamos por sus ids en interrogatorio

In [27]:
query = '''
SELECT a.person_id, a.name, b.*
FROM get_fit_now_member AS a
JOIN interview AS b
ON a.person_id = b.person_id
WHERE b.person_id IN (67318,28819)
'''
df = pd.read_sql_query(query, conn)
df

,person_id,name,person_id,transcript
0,67318,Jeremy Bowers,67318,I was hired by a woman with a lot of money. I ...


In [28]:
print(df["transcript"][0])

I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



### Paso 8: Buscamos en drivers license con la info que da Jeremy

In [29]:
query = '''
SELECT *
FROM drivers_license
WHERE hair_color = "red"
    AND gender = "female"
    AND car_make = "Tesla"
'''
pd.read_sql_query(query, conn)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


### Paso 9: Nuestras sospechosas

In [30]:
query = '''
SELECT a.id, b.name, b.ssn
FROM drivers_license AS a
JOIN person AS b
ON a.id = b.license_id
WHERE a.id IN (202298,291182,918773)
'''
pd.read_sql_query(query, conn)

,id,name,ssn
0,202298,Miranda Priestly,987756388
1,291182,Regina George,337169072
2,918773,Red Korb,961388910


### Paso 10: Consultamos eventos de Facebook y llegamos a la solución

In [33]:
query = '''
SELECT a.id, a.name, b.*
FROM person AS a
JOIN facebook_event_checkin AS b
ON a.id = b.person_id
WHERE a.id IN (78881,99716,90700)
'''
pd.read_sql_query(query, conn)

,id,name,person_id,event_id,event_name,date
0,99716,Miranda Priestly,99716,1143,SQL Symphony Concert,20171206
1,99716,Miranda Priestly,99716,1143,SQL Symphony Concert,20171212
2,99716,Miranda Priestly,99716,1143,SQL Symphony Concert,20171229


In [ ]:
cursor = conn.cursor()
cursor.execute('INSERT INTO solution VALUES (1, "Miranda Priestly")')
conn.commit()

In [39]:
print(pd.read_sql_query("SELECT value FROM solution", conn)["value"][0])

Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!


## 💡 Enfoque 2

### Paso 1: Filtrar la escena del crimen por fecha, tipo y lugar

In [40]:
query = '''
SELECT * FROM crime_scene_report
WHERE date = "20180115"
    AND type = "murder" 
    AND city = "SQL City"
'''
df = pd.read_sql_query(query, conn)
df

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [41]:
print(df["description"][0])

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


### Paso 2: Buscamos a los dos testigos

In [47]:
query = '''
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC
'''
pd.read_sql_query(query,conn).iloc[0,:]

id                               14887
name                    Morty Schapiro
license_id                      118009
address_number                    4919
address_street_name    Northwestern Dr
ssn                          111564949
Name: 0, dtype: object

In [48]:
query = '''
SELECT *
FROM person
WHERE name LIKE '%Annabel%'
AND address_street_name = "Franklin Ave"
'''
pd.read_sql_query(query,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


### Paso 3: Buscamos sus testimonios en los interrogatorios

In [51]:
query = '''
SELECT *
FROM interview
WHERE person_id IN ("14887", "16371")
'''
df = pd.read_sql_query(query,conn)
df

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


In [56]:
print(df["transcript"][0])
print("-"*220)
print(df["transcript"][1])

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


### Paso 4: Buscamos sospechosos cruzando pistas de las declaraciones

In [57]:
query = '''
SELECT *
FROM get_fit_now_check_in
WHERE membership_id LIKE "%48Z%"
    AND check_in_date = "20180109";
'''
pd.read_sql_query(query,conn)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [ ]:
query = '''
SELECT *
FROM drivers_license
WHERE gender = "male"
	AND plate_number LIKE '%H42W%';
'''
pd.read_sql_query(query,conn)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


### Paso 5: Buscamos las personas asociadas a los coches

In [59]:
query = '''
SELECT *
FROM person
WHERE license_id IN ("423327", "664760")
'''
pd.read_sql_query(query,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


### Paso 6: Comprobamos si son miembros del gimnasio

In [60]:
query = '''
SELECT *
FROM get_fit_now_member
WHERE person_id IN ("51739", "67318")
'''
pd.read_sql_query(query,conn)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


### Paso 7: Comprobamos solución

In [61]:
cursor.execute('INSERT INTO solution VALUES (1, "Jeremy Bowers")')
conn.commit()
print(pd.read_sql_query("SELECT value FROM solution", conn)["value"][0])

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.


Intentemos buscar el cerebro en dos querys, como se pide:

### Paso 8: Imprescindible, leer la declaración de Jeremy

In [63]:
query = '''
SELECT *
FROM interview
WHERE person_id = 67318
'''
df = pd.read_sql_query(query,conn)
df

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


In [64]:
print(df["transcript"][0])

I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



### Paso 9: Encontrar al cerebro con todas las pistas

In [67]:
#Para hacerlo en una sola query anidamos consultas
query = '''
SELECT *
FROM person
WHERE license_id IN (
    SELECT id
    FROM drivers_license
    WHERE gender = "female"
      AND car_make = "Tesla"
      AND car_model = "Model S"
      AND hair_color = "red"
      AND height BETWEEN 64 AND 66
)
AND id IN (
    SELECT person_id
    FROM facebook_event_checkin
    WHERE event_name = "SQL Symphony Concert"
      AND date LIKE "201712%"
    GROUP BY person_id
    HAVING COUNT(*) >= 3
)
'''
pd.read_sql_query(query,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


Y, efectivamente, llegamos a lo mismo:


> 🔎 **Miranda Priestly** es el cerebro y **Jeremy Bowers** el asesino.
>
> En el primer enfoque pasamos por la entrevista de Jeremy para llegar directamente a Miranda.
>
> En el segundo filtramos el asesino siguiendo los testimonios de testigos, y desde ahí llegamos en dos querys a Miranda como nos indica al comprobar la solución de Jeremy